## Tensorflow使用tf-serving实现在线预估服务

背景：
* tensorflow提供了tf-serving功能，用于高性能的线上tf预估服务
* 使用tf-serving的最佳方式，是使用docker进行模型加载启动服务

演示步骤：
1. 将Keras模型导出成tf-serving需要的格式
2. 使用docker拉取tf-serving镜像
3. 使用docker命令加载模型启动服务
4. 使用HTTP/GRPC即可访问服务

In [1]:
import pandas as pd
import tensorflow as tf

### 1. 准备和训练一个模型

#### 读取数据

In [2]:
df = pd.read_csv("./datas/heart/heart.csv")

# 把thal列变成数字编码
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df['thal'].cat.codes

# 要预测的目标，这是个二分类问题
target = df.pop('target')

#### 构建dataset

In [3]:
# 构建dataset，其实是把pandas数据转换成numpy数组进行转换的
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))
# Shuffle and batch the dataset.
train_dataset = dataset.shuffle(len(df)).batch(4)

In [4]:
# 用于一会的测试
for x, y in train_dataset.take(1):
    input_data = x.numpy()
    print(input_data)

[[ 44.    1.    2.  120.  220.    0.    0.  170.    0.    0.    1.    0.
    3. ]
 [ 60.    1.    4.  145.  282.    0.    2.  142.    1.    2.8   2.    2.
    4. ]
 [ 51.    1.    3.  110.  175.    0.    0.  123.    0.    0.6   1.    0.
    3. ]
 [ 59.    1.    0.  164.  176.    1.    0.   90.    0.    1.    1.    2.
    0. ]]


#### 搭建训练模型

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, input_shape=(df.shape[1],)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer='adam',
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=['accuracy'])

model.fit(train_dataset, epochs=10)

Epoch 1/10
76/76 [==============================] - 0s 1ms/step - loss: 39.1831 - accuracy: 0.6733
Epoch 2/10
76/76 [==============================] - 0s 2ms/step - loss: 9.4004 - accuracy: 0.5446
Epoch 3/10
76/76 [==============================] - 0s 1ms/step - loss: 7.1186 - accuracy: 0.5776
Epoch 4/10
76/76 [==============================] - 0s 1ms/step - loss: 5.1598 - accuracy: 0.6040
Epoch 5/10
76/76 [==============================] - 0s 2ms/step - loss: 3.4348 - accuracy: 0.5941
Epoch 6/10
76/76 [==============================] - 0s 865us/step - loss: 2.1056 - accuracy: 0.6469
Epoch 7/10
76/76 [==============================] - 0s 1ms/step - loss: 1.3750 - accuracy: 0.6865
Epoch 8/10
76/76 [==============================] - 0s 937us/step - loss: 0.8724 - accuracy: 0.7261
Epoch 9/10
76/76 [==============================] - 0s 1ms/step - loss: 0.6434 - accuracy: 0.7525
Epoch 10/10
76/76 [==============================] - 0s 1ms/step - loss: 0.6946 - accuracy: 0.8086


### 2. 以tf-serving格式保存模型

In [6]:
input_data

array([[ 44. ,   1. ,   2. , 120. , 220. ,   0. ,   0. , 170. ,   0. ,
          0. ,   1. ,   0. ,   3. ],
       [ 60. ,   1. ,   4. , 145. , 282. ,   0. ,   2. , 142. ,   1. ,
          2.8,   2. ,   2. ,   4. ],
       [ 51. ,   1. ,   3. , 110. , 175. ,   0. ,   0. , 123. ,   0. ,
          0.6,   1. ,   0. ,   3. ],
       [ 59. ,   1. ,   0. , 164. , 176. ,   1. ,   0. ,  90. ,   0. ,
          1. ,   1. ,   2. ,   0. ]])

In [7]:
import pandas as pd
pd.DataFrame(input_data).to_json(orient='values')

'[[44.0,1.0,2.0,120.0,220.0,0.0,0.0,170.0,0.0,0.0,1.0,0.0,3.0],[60.0,1.0,4.0,145.0,282.0,0.0,2.0,142.0,1.0,2.8,2.0,2.0,4.0],[51.0,1.0,3.0,110.0,175.0,0.0,0.0,123.0,0.0,0.6,1.0,0.0,3.0],[59.0,1.0,0.0,164.0,176.0,1.0,0.0,90.0,0.0,1.0,1.0,2.0,0.0]]'

In [8]:
tf.keras.models.save_model(
    model,
    "./tf_serving_model/heart_model/1",
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./tf_serving_model/heart_model/1/assets


### 3. 以下操作均为shell命令

#### 下载docker镜像的命令：
```
docker pull tensorflow/serving
docker images
```

#### 启动服务的shell命令
```
model_dir="/home/pss/workbench/ant-learn-tensorflow/tf_serving_model/heart_model"
sudo docker run -p 8501:8501 \
  --mount type=bind,source=${model_dir},target=/models/heart_model \
  -e MODEL_NAME=heart_model -t tensorflow/serving
```

#### 访问接口的命令：
```
curl -d '{"instances": [[62.0,0.0,3.0,130.0,263.0,0.0,0.0,97.0,0.0,1.2,2.0,1.0,4.0],[43.0,1.0,4.0,150.0,247.0,0.0,0.0,171.0,0.0,1.5,1.0,0.0,3.0],[76.0,0.0,3.0,140.0,197.0,0.0,1.0,116.0,0.0,1.1,2.0,0.0,3.0]]}' -X POST http://192.168.0.119:8501/v1/models/heart_model:predict
```